In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    "unsloth/gemma-2-9b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"  # Qwen 2.5 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.6 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
import kagglehub
path = kagglehub.dataset_download("thedevastator/mental-health-chatbot-pairs")

100%|██████████| 56.8k/56.8k [00:00<00:00, 29.7MB/s]

Extracting files...


In [ ]:
print(path)

/root/.cache/kagglehub/datasets/thedevastator/mental-health-chatbot-pairs/versions/2


In [ ]:
import pandas as pd
import os
os.listdir(path)
# print(df.head())
df = pd.read_csv(os.path.join(path, "train.csv"))
print(df.head())

df.to_csv("train.csv", index= False)   #to load the csv

                                                text
0  <HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1  <HUMAN>: What are symptoms of panic attack vs....
2  <HUMAN>: What are the types of Mental Illness?...
3  <HUMAN>: What does mental-illness mean?\n<ASSI...
4  <HUMAN>: How can you treat mental illness?\n<A...


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="train.csv",
    split="train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.column_names

['text']

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)

def formatting_prompts_func(examples):
    texts = []

    for conv in examples["text"]:
        if "<HUMAN>:" in conv and "<ASSISTANT>:" in conv:

            # Extract human nd assistant
            human = conv.split("<HUMAN>:")[1].split("<ASSISTANT>:")[0].strip()
            assistant = conv.split("<ASSISTANT>:")[1].strip()

            # Build chat-style conversation
            convo = [
                {
                    "role": "system",
                    "content": (
                        "You are a supportive and empathetic mental health assistant."
                    )
                },
                {
                    "role": "user",
                    "content": human
                },
                {
                    "role": "assistant",
                    "content": assistant
                }
            ]

            # Apply Unsloth chat template
            text = tokenizer.apply_chat_template(
                convo,
                tokenize=False,
                add_generation_prompt=False
            )

            texts.append(text)

    return {"text": texts}

In [ ]:
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0]["text"])


<|im_start|>system<|im_sep|>You are a supportive and empathetic mental health assistant.<|im_end|><|im_start|>user<|im_sep|>What is a panic attack?<|im_end|><|im_start|>assistant<|im_sep|>Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.<|im_end|>


In [ ]:
dataset[5]["text"]

'<|im_start|>system<|im_sep|>You are a supportive and empathetic mental health assistant.<|im_end|><|im_start|>user<|im_sep|>Are patients with schizophrenia violent?<|im_end|><|im_start|>assistant<|im_sep|>Due to incorrect representation of the disease in media & books, there is a myth that schizophrenics are violent. The truth is most schizophrenics are docile and keep to themselves. The schizophrenics who have demonstrated bursts of violence are either in an acute stage of psychosis or are abusing an addictive substance.<|im_end|>'

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

Map (num_proc=6):   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system<|im_sep|>You are a supportive and empathetic mental health assistant.<|im_end|><|im_start|>user<|im_sep|>Are patients with schizophrenia violent?<|im_end|><|im_start|>assistant<|im_sep|>Due to incorrect representation of the disease in media & books, there is a myth that schizophrenics are violent. The truth is most schizophrenics are docile and keep to themselves. The schizophrenics who have demonstrated bursts of violence are either in an acute stage of psychosis or are abusing an addictive substance.<|im_end|>'

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 172 | Num Epochs = 2 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 65,536,000 of 14,725,043,200 (0.45% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.429000
2,1.063900
3,1.476600
4,1.217100
5,1.398400
6,0.932600
7,1.158600
8,1.329100
9,1.162700
10,0.769400


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "What is a panic attack?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1
)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user<|im_sep|>What is a panic attack?<|im_end|><|im_start|>assistant<|im_sep|>A panic attack is a sudden and intense feeling of fear or anxiety that can occur without warning. Panic attacks can be very frightening and can feel like a heart attack or other serious medical emergency. Symptoms of a panic attack can include a rapid heartbeat, sweating, trembling, shortness of breath, chest pain, nausea, d']

We used TextStreamer for continuous inference - so you can see the generation token by token, instead of waiting the whole time!




In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How to deal with loneliness"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

Dealing with loneliness can be challenging, but there are several strategies you can try to cope with it:

1. Reach out to others: Connect with friends, family, or acquaintances. Reach out to someone you trust and share your feelings. Sometimes, just talking about your emotions can make a big difference.

2. Join social groups or clubs: Look for local clubs, organizations, or groups that align with your interests. This can be a great way to meet new people and make friends who share similar hobbies or passions.

3. Volunteer: Consider volunteering for a cause you care about. It's a meaningful way to connect with others and


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
model.push_to_hub("Finetuned-Phi_4-Conversational")
tokenizer.push_to_hub("Finetuned-Phi_4-Conversational")


README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 62.9kB /  262MB            

Saved model to https://huggingface.co/Finetuned-Phi_4-Conversational


In [ ]:
model.push_to_hub_merged(
    "prranjal/16-bit-Phi_4-Conversational",
    tokenizer,
    save_method="merged_16bit"
)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00006.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/6 [00:00<?, ?it/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0006-of-00006.safetensors:   1%|          | 41.9MB / 4.62GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 6/6 [25:59<00:00, 259.91s/it]


Unsloth: Merge process complete. Saved to `/content/prranjal/16-bit-Phi_4-Conversational`
